In [1]:
from langchain_community.chat_models import ChatOpenAI
import dotenv
import os
import joblib
from discussion_agents.cog.agent.expel import ExpeLAgent

dotenv.load_dotenv("../.env")
openai_api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", openai_api_key=openai_api_key)
hotpotqa = joblib.load('../../../tests/assets/hotpotqa/hotpot-qa-distractor-sample.joblib')


In [ ]:
agent = ExpeLAgent(
    llm=llm,
    self_reflect_llm=llm,
    action_llm=llm
)

In [ ]:
agent.gather_experience(
    questions=hotpotqa.question.values[:1],
    keys=hotpotqa.answer.values[:1]
)

In [ ]:
agent.insight_memory.insights

In [ ]:
import joblib 
from discussion_agents.cog.modules.memory.expel import ExpeLExperienceMemory

hotpotqa = joblib.load('../../../tests/assets/hotpotqa/hotpot-qa-distractor-sample.joblib')
experiences = joblib.load('../../../tests/assets/expel/expel_experiences_10_fake.joblib')
memory = ExpeLExperienceMemory(experiences)

In [ ]:
queries = {
    "task": 'The creator of "Wallace and Gromit" also created what animation comedy that matched animated zoo animals with a soundtrack of people talking about their homes? ',
    "thought": 'Thought: I should try a different approach. Let me search for press releases, industry news sources, or announcements specifically related to the name change and new acronym for VIVA Media AG in 2004. By focusing on more specialized sources, I may be able to find the accurate information needed to answer the question correctly. ',
    "other": "Some other query."
}

memory_dict = memory.load_memories(query=queries["task"])['fewshots']

In [ ]:
print("\n\n".join(memory_dict + ["(END OF EXAMPLES)"]))

In [ ]:
RULE_PREFIX = """
The following are some experience you gather on a similar task of question answering using Wikipedia API. Use these as references to help you perform this task:
"""
print(RULE_PREFIX)

In [ ]:
from discussion_agents.cog.modules.memory.expel import ExpeLInsightMemory

insights = [('If the initial search does not provide information related to the specific question, try to explore different angles or sources to gather relevant information that may lead to the answer. Consider refining the search query with more specific and relevant keywords to narrow down the search results and find the desired information effectively.',
  5),
 ('If unable to find the answer after exploring different search options, consider reaching out to additional sources or experts for assistance in obtaining the required information.',
  2),
 ('When searching for specific information, try to include relevant keywords related to the topic to narrow down the search results and find the desired information effectively.',
  2),
 ('When searching for specific information, consider verifying the credibility of the sources to ensure the accuracy of the information retrieved.',
  2),
 ('When faced with a search query, consider utilizing different search engines or platforms to gather information and find the desired answer effectively.',
  1)]

insights = [{"insight": insight[0], "score": insight[1]} for insight in insights]
memory = ExpeLInsightMemory(insights=insights, max_num_insights=5, leeway=1)

In [ ]:
memory.load_memories()['insights']

In [ ]:
insights = memory.load_memories()['insights']

c = "".join([f"{i}. {insight['insight']}\n" for i, insight in enumerate(insights, 1)])

In [ ]:
print(a)

In [ ]:
print(c)

In [ ]:
insights = memory.load_memories()['insights']

a = "The following are some experience you gather on a similar task of question answering using Wikipedia API. Use these as references to help you perform this task:\n"
for i, insight in enumerate(insights, 1):
    a += f"{i}. {insight['insight']}\n"

In [ ]:
b = "\n\n".join(memory_dict + ["(END OF EXAMPLES)"]) + "\n"
b += RULE_PREFIX
c = "".join([f"{i}. {insight['insight']}\n" for i, insight in enumerate([], 1)])
b += c

In [ ]:
print(b)

In [ ]:
EXPEL_REFLEXION_REACT_INSTRUCTION = """
Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be three types: 
(1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
(2) Lookup[keyword], which returns the next sentence containing keyword in the last passage successfully found by Search.
(3) Finish[answer], which returns the answer and finishes the task.
You have a maximum of {max_steps} steps.

Here are some examples:
{examples}

{reflections}

Question: {question}{scratchpad}
"""

examples = "\n\n".join(memory_dict) + "\n\n" + "(END OF EXAMPLES)" + "\n\n"
print(examples)

In [ ]:
agent = ExpeLAgent(
    llm=llm,
    self_reflect_llm=llm,
    action_llm=llm
)

In [ ]:
agent.insight_memory.insights

In [ ]:
agent.gather_experience(
    questions=hotpotqa.question.values[:1],
    keys=hotpotqa.answer.values[:1]
)